<a href="https://colab.research.google.com/github/AUT-Student/SML-Project/blob/main/SML_Project_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [39]:
overall_dataset = pd.read_csv("/content/Overall.csv")

In [40]:
overall_dataset["Return"] = overall_dataset["Close"].diff()/overall_dataset["Close"]

In [5]:
overall_dataset["Return"].autocorr(lag=5)

-0.029116582025897918

In [7]:
for lag in range(1, 6):
  autocorrelation = overall_dataset["Return"].autocorr(lag=lag)
  print(f"Lag = {lag}, Autocorrelation = {autocorrelation}")

Lag = 1, Autocorrelation = 0.3420962070256291
Lag = 2, Autocorrelation = 0.026094104901236134
Lag = 3, Autocorrelation = 0.06856595465315196
Lag = 4, Autocorrelation = -0.008159864385430154
Lag = 5, Autocorrelation = -0.029116582025897918


In [8]:
!gdown 1mFilhzDbZqb4xC6ifBLPjha1Svmqcr9X

Downloading...
From: https://drive.google.com/uc?id=1mFilhzDbZqb4xC6ifBLPjha1Svmqcr9X
To: /content/SML_Project_Dataset.zip
100% 400k/400k [00:00<00:00, 79.7MB/s]


In [9]:
!unzip /content/SML_Project_Dataset.zip

Archive:  /content/SML_Project_Dataset.zip
  inflating: Kekhak.csv              
  inflating: Foolad.csv              
  inflating: Shasta.csv              
  inflating: Khodro.csv              
  inflating: Atlas.csv               


In [10]:
atlas_dataset = pd.read_csv("/content/Atlas.csv")
foolad_dataset = pd.read_csv("/content/Foolad.csv")
kekhak_dataset = pd.read_csv("/content/Kekhak.csv")
khodro_dataset = pd.read_csv("/content/Khodro.csv")
shasta_dataset = pd.read_csv("/content/Shasta.csv")

In [11]:
datasets = {
    "Atlas": atlas_dataset,
    "Foolad": foolad_dataset,
    "Kekhak": kekhak_dataset,
    "Khodro": khodro_dataset,
    "Shasta": shasta_dataset
}

In [12]:
for name in datasets:
  dataset = datasets[name]
  dataset["Return"] = dataset["<CLOSE>"].diff()/dataset["<CLOSE>"]  
  dataset = dataset[(dataset["<DTYYYYMMDD>"] >= 20210101) & (dataset["<DTYYYYMMDD>"] <= 20221230)]
  datasets[name] = dataset

In [13]:
datasets["Atlas"]

,<TICKER>,<DTYYYYMMDD>,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>,Return
21,ToseAtlasMofid.ETF,20221228,311010.0,340452.0,310005.0,326194.0,598795040036,1835702,2326,D,309502.0,328011.0,0.058545
22,ToseAtlasMofid.ETF,20221226,305100.0,313993.0,303000.0,309502.0,243277589501,786030,1214,D,306832.0,310101.0,-0.053932
23,ToseAtlasMofid.ETF,20221225,305001.0,310001.0,300010.0,306832.0,164293171128,535450,1140,D,301381.0,308020.0,-0.008702
24,ToseAtlasMofid.ETF,20221224,292995.0,304985.0,292900.0,301381.0,200054004059,663790,1047,D,288344.0,304675.0,-0.018087
25,ToseAtlasMofid.ETF,20221221,286514.0,290000.0,286514.0,288344.0,58642929251,203378,447,D,287756.0,288968.0,-0.045213
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,ToseAtlasMofid.ETF,20210106,210000.0,210000.0,206000.0,207916.0,15222131644,73213,358,D,214140.0,207800.0,-0.011283
493,ToseAtlasMofid.ETF,20210105,215499.0,215499.0,213000.0,214140.0,44795676126,209189,344,D,213410.0,213210.0,0.029065
494,ToseAtlasMofid.ETF,20210104,211500.0,216000.0,211500.0,213410.0,148766239899,697092,439,D,210450.0,214500.0,-0.003421
495,ToseAtlasMofid.ETF,20210103,205555.0,215000.0,205555.0,210450.0,61317069222,291361,538,D,205931.0,214300.0,-0.014065


In [16]:
overall_dataset

,J-Date,Date,Open,High,Low,Close,Adj Close,Volume,Return
0,1399-10-13,2021-01-02,1389700.0,1389700.0,1354110.0,1354114.0,1354147.1,6065577245,NaN
1,1399-10-14,2021-01-03,1347700.0,1357900.0,1345820.0,1355475.0,1355351.2,8925080297,0.001004
2,1399-10-15,2021-01-04,1361540.0,1375350.0,1361540.0,1374501.0,1374483.2,8791746228,0.013842
3,1399-10-16,2021-01-05,1372480.0,1372480.0,1350210.0,1350211.0,1350233.2,8904611865,-0.017990
4,1399-10-17,2021-01-06,1342160.0,1342160.0,1310450.0,1310709.0,1310755.8,8503250697,-0.030138
...,...,...,...,...,...,...,...,...,...
473,1401-10-03,2022-12-24,1500750.0,1538000.0,1500750.0,1538003.0,1538003.2,10103385881,0.027179
474,1401-10-04,2022-12-25,1547140.0,1563560.0,1547140.0,1563567.0,1563567.4,10776995540,0.016350
475,1401-10-05,2022-12-26,1566820.0,1576410.0,1563160.0,1576397.0,1576396.8,9446689906,0.008139
476,1401-10-07,2022-12-28,1588820.0,1647310.0,1588820.0,1647314.0,1647314.5,15175183890,0.043050


In [41]:
overall_dataset["<DTYYYYMMDD>"] = overall_dataset["Date"].apply(lambda x:int(x[:4]+x[5:7]+x[8:10]))

In [42]:
overall_dataset = overall_dataset[["<DTYYYYMMDD>", "Return"]]

In [49]:
for name in datasets:
  common_dataset = datasets[name][["<DTYYYYMMDD>", "Return"]]\
                  .set_index("<DTYYYYMMDD>")\
                  .join(overall_dataset\
                        .set_index("<DTYYYYMMDD>"),
                        how="inner", lsuffix='_share', rsuffix='_overall')

  correlation = common_dataset["Return_share"].corr(common_dataset["Return_overall"])
  print(f"{name}: {correlation}")

Atlas: -0.11624312213988305
Foolad: -0.16704998818279018
Kekhak: -0.15260274065831042
Khodro: -0.19881093525313684
Shasta: -0.07223098356120301


In [50]:
common_dataset

,Return_share,Return_overall
<DTYYYYMMDD>,,
20221228,0.043265,0.043050
20221226,-0.048454,0.008139
20221225,-0.007269,0.016350
20221224,-0.010493,0.027179
20221221,-0.046103,0.002744
...,...,...
20210106,0.038226,-0.030138
20210105,0.039648,-0.017990
20210104,0.043091,0.013842
